Using the LJ Dataset to train the model presents us with several symbols that cannot be represented in the Roman alphanumeric notation. The unicode library is used in the project to transcribe these characters to ASCII.

In [ ]:
!pip install -q unidecode librosa inflect

     |████████████████████████████████| 235 kB 26.6 MB/s 


In [ ]:
%%bash

apt-get update
apt-get install -y libsndfile1

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 

#WIP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, num_chars, padding_idx, embedding_dim=512, encoder_num_convs=2, kernel_size=8, hidden_size=256, bidirectional=True):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(num_chars, embedding_dim, padding_idx=padding_idx)
        padding = (kernel_size - 1) // 2
        convs = []

        for _ in range(encoder_num_convs):
            convs += [Convolutional_Block(embedding_dim, embedding_dim, kernel_size, padding, 'relu')]

        self.convs = nn.Sequential(*convs)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=1, batch_first=True, bidirectional=bidirectional)
    
    #
    def forward(self, text_padded, input_lengths):
        x = self.embedding(text_padded)
        x = x.transpose(1, 2)
        x = self.convs(x)
        x = x.transpose(1, 2)
        total_length = x.size(1)
        packed_input = pack_padded_sequence(x, input_lengths, batch_first=True)
        self.rnn.flatten_parameters()
        packed_output, _ = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=total_length)
        return output


class Convolutional_Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, nonlinear=None):
        super(Convolutional_Block, self).__init__()
        conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding)
        norm = nn.BatchNorm1d(out_channels)
        
        dropout = nn.Dropout(p=0.2)
        if nonlinear == 'relu':
            relu = nn.ReLU()
            self.net = nn.Sequential(conv1d, norm, relu, dropout)

        elif nonlinear == 'tanh':
            tanh = nn.Tanh()
            self.net = nn.Sequential(conv1d, norm, tanh, dropout)

        else:
            self.net = nn.Sequential(conv1d, norm, dropout)
    def forward(self, x):
        output = self.net(x)
        return output


if __name__ == "__main__":
    torch.manual_seed(555)
    N, _T, T = 3, 6, 8
    num_chars, padding_idx, feature_dim = 10, 0, 5
    text_padded = torch.randint(num_chars, (N, _T)).long()
    input_lengths = torch.LongTensor([_T, _T-2, _T-4])
    feat_padded = torch.randint(5, (N, T, feature_dim))
    feat_lengths = torch.LongTensor([T, T-1, T-3])
    print(input_lengths)
    print(feat_lengths)

    # Future Prdiction point not set
    

In [ ]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16', pretrained=True)
tacotron2 = tacotron2.to('cuda')

waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16', pretrained=True)
waveglow = waveglow.to('cuda')

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/waveglow/model.py:55: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be r

In [9]:
text = "Choppy but Works. That is it for the update."

In [10]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [12]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22000

In [13]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)